In [1]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [17]:
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
    losses,
    evaluation
)
from datasets import Dataset, concatenate_datasets
from peft import LoraConfig, TaskType, get_peft_model
import ast

In [18]:
def load_triplet_data(path):
    df = pd.read_csv(path)
    df['negative'] = df['negatives'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Exploding the 'negative' column to create multiple rows
    df_expanded = df.explode('negative').reset_index(drop=True)

    return Dataset.from_dict({
        "anchor": df_expanded['anchor'].tolist(),
        "positive": df_expanded['positive'].tolist(),
        "negative": df_expanded['negative'].tolist()
    })

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
data_path = '/content/drive/MyDrive/data/'


train_data = load_triplet_data(data_path + "train.csv")
test_data = load_triplet_data(data_path + "test.csv")
val_data = load_triplet_data(data_path + "dev.csv")

In [22]:
train_data[0]

{'anchor': 'የታክስ ገቢ ከ2010-2012 በመቶኛ የምን ያህል መጠን እድገት አሳየ?',
 'positive': 'ጠቅላይ ሚኒስትር ዐቢይ አሕመድ ከ2010 ጀምሮ በፋይናንሱ ዘርፍ ስኬታማ ለውጦች መመዝገባቸውን ገለጹ፡፡ ጠቅላይ ሚኒስትር ዐቢይ የፋይናንስ ዘርፍ ዐበይት ስኬቶች በሚል በማህበራዊ ትስስር ገፃቸው ላይ እንዳስታወቁት የታክስ ገቢ በ2010 ከነበረበት 229 ቢሊየን ብር በ2012 የ36 በመቶ ጭማሪ በማሳየት 311 ቢሊየን ማድረስ ተችሏል።',
 'negative': 'ፋሲለደስ ዓፄ ፋሲለደስ ወይም ዓፄ ፋሲል (የዙፋን ስማቸው ዓለም ሰገድ)  ከአባታቸው አፄ ሱሰኒዮስ  እና እናታቸው ልዕልት ስልጣነ ምገሴ  በመገዛዝ፣ ሸዋ ህዳር 10፣ 1603  (እ.ኤ.አ) ተወለዱ። የነገሱበትም ዘመን ከ1632  እስከ ጥቅምት 18, 1667 (እ.ኤ.አ) ነበር። በስረፀ ክርስቶስ በተመራው አመፅ ምክንያት በ1630 ፋሲለደስ ለንግስና ቢበቃም፣ ዘውዱን ግን እስከ 1632 አልጫነም ነበር። ሲመተ ንግስናው በ1632 እንደተገባደደ የመጀመሪያው ስራው የተዋህዶ ቤ/ክርስቲያንን የቀድሞው ቁመና መመለስና የካቶሊኮችን መሬት በመቀማት ከደንካዝ በማባረር በፍሪሞና እንዲወሰኑ ማድረግ ነበር። ወዲያውም በማከታተል ከግብፅ አገር አዲስ ጳጳስ እንዲላክለት በማድረግ በአባቱ ዘመን እንዲደበዝዝ ተደርጎ የነበረውን የግብፅና ኢትዮጵያ አብያተ ክርስቲያናት ግንኙነት እንዲጸና አደረገ።  በኬኒያ የሚገኘው የሞምባሳ ወደብ በፖርቱጋሎች መደብደቡን ሲሰማ፣ የሮማው ፓፓ ከበስትጀርባው ያለበት ሴራ ነው በማለት በምድሩ የነበሩትን የካቶሊክ ጀስዊቶች በመሰብሰብ አባረራቸው። አፄ ፋሲል አዘዞ ተብላ በምትታወቀው ከጎንደር ከተማ 5 ማይል ርቃ በምትገኘው ከተማ ጥቅምት18፣1667 እ.ኤ.አ. (ጥቅምት 10፣ 1660) ከዚህ

In [23]:
len(train_data), len(test_data), len(val_data)

(6715, 1440, 2520)

In [24]:
# Let's halve the test and val data and add them to the training data
half_val_size = len(val_data) // 2
half_test_size = len(test_data) // 2

# Select the first half of validation and test datasets
val_data_half = val_data.select(range(half_val_size))
val_data = val_data.select(range(half_val_size, len(val_data))) # the new val data
test_data_half = test_data.select(range(half_test_size))
test_data = test_data.select(range(half_test_size, len(test_data))) # the new test data

train_data = concatenate_datasets([train_data, val_data_half, test_data_half])

print(len(train_data), len(test_data), len(val_data))

8695 720 1260


In [25]:
import random

def shuffle_dataset(dataset, seed=43):
  """Shuffles the given dataset in-place."""
  random.seed(seed) # for reproducibility
  items = list(dataset)
  random.shuffle(items)
  return Dataset.from_list(items)

train_data = shuffle_dataset(train_data)
test_data = shuffle_dataset(test_data)
val_data = shuffle_dataset(val_data)
train_data[0]

{'anchor': 'በኢትዮጰጵያ የዲጂታል ኢኮኖሚውን ያጠናክራል ተብሎ የተቋቋመው የመረጃ ቁዋት ማዕከል ምን ይባላል?',
 'positive': 'የዲጂታል ኢኮኖሚውን የሚያጠናክር ሬድፎክስ የተሰኘ የመረጃ ቋት ማዕከል ይፋ ሆነ፡፡ የገንዘብ ሚኒስትር ዴኤታ ዶክተር እዮብ ተካልኝ መረጃ አዲሱ ነዳጅ መሆኑን ገልጸው፣ የዲጂታል መሰረተ ልማቱ መጠናከር በጤና፣ በትምህርት፣ በግብርና እና በሌሎች መስኮች ላይ አስተዋጽኦን ያሳርፋል ብለዋል፡፡ ሬድፎክስ ለረጅም ዓመታት በቴሌኮም ዘርፍና በመረጃ ቴክኖሎጂ ልምድ ባካበቱ የኢትዮጵያ ዳያስፖራዎች የተቋቋመ መሆኑም ተገልጿል፡፡ ሚኒስትር ዲኤታው ዳያስፖራዎቸ በዘርፉ ላይ በመሰማራታቸው ያላቸውን አድናቆት መግለጻቸውን ከገንዘብ ሚኒስቴር ያገኘነው መረጃ ያመለክታል፡፡',
 'negative': ' የቻይና ተመራማሪዎች እጅግ በጣም ከፍተኛ በሆነ የሙቀት ወቅት የአውሮፕላኖችን ሁኔታ መከታተል የሚያስችል ዘዴ ይፋ አደረጉ። አዲሱ ቴክኖሎጂ አውሮፕላኖች ከ800 እስከ 1 ሺህ 300 ዲግሪ ሴሊሺየስ ባለው የሙቀት መጠን ውስጥ ሲሰሩ በአውሮፕላኖቹ ውስጥ የሚከሰተውን መዋቅራዊ ብልሽት መለየት የሚያስችል መሆኑን ተመራማሪዎቹ ገልጸዋል። ይህም እጅግ ከፍተኛ የሆነ ሙቀትና ቅዝቃዜ ባለበት የአየር ንብረት ውስጥ የሚንቀሳቀሱ አውሮፕላኖችን አጠቃላይ ሁኔታ ለመከታተል የሚያግዝ ነው ተብሏል። ከዚህ ባለፈም አውሮፕላኑ በከፍተኛ ፍጥነት በሚበርበት ወቅት የሚያጋጥመውን ከፍተኛ ሙቀት ተቋቁሞ በረራውን መቀጠል ስለመቻል አለመቻሉም ለመወሰን እንደሚያግዝም ከተመራማሪዎቹ አንዱ የሆኑት ዋንግ ዚዮንግ ተናግረዋል። የአውሮፕላኖች አሰራር እጅግ ከፍተኛ በሆነ የሙቀት መጠን ውስጥ የሚደረግን በረራ መቋቋም ስለመቻል አለመቻሉ በመወሰን አዲስ ለሚሰራ የአውሮፕላን ንድፍ እንደ ማጣቀሻ

In [39]:
# Option 1: Base XLM-R with PEFT

# model_name = "sentence-transformers/static-similarity-mrl-multilingual-v1"
# model_name= "sentence-transformers/multi-qa-mpnet-base-cos-v1" - FAILURE
model_name = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
# model_name = "Pyjay/sentence-transformers-multilingual-snli-v2-500k" - Works but is too big

model = SentenceTransformer(
    model_name,
    model_card_data=SentenceTransformerModelCardData(
        language="am",
        license="mit",
        model_name=f"{model_name} adapter finetuned on Amharic QA triplets",
    ),)



In [27]:
sample_text = "ውክፔዲያ ዓለምዓቀፍ የዕውቀት ማከማቻ ድረ-ገጽ ነው። ውክፔዲያ የጋራ ነው።"
tokens = model.tokenizer.tokenize(sample_text)
print("Tokenization:", tokens) # We might need to improve this later on, but it's working for now

Tokenization: ['▁', 'ው', 'ክ', 'ፔ', 'ዲያ', '▁ዓለም', 'ዓ', 'ቀ', 'ፍ', '▁የ', 'ዕውቀት', '▁ማ', 'ከ', 'ማ', 'ቻ', '▁ድረ', '-', 'ገ', 'ጽ', '▁ነው።', '▁', 'ው', 'ክ', 'ፔ', 'ዲያ', '▁የጋራ', '▁ነው።']


In [28]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate_model(model, model_name, anchor, candidates):
    print(f"\n{'='*30} {model_name} {'='*30}")
    anchor_emb = model.encode([anchor])
    candidate_embs = model.encode(candidates)

    # Calculate similarities
    similarities = cosine_similarity(anchor_emb, candidate_embs)[0]

    # Print ranked results
    ranked_indices = np.argsort(-similarities)
    print("Original Sentence: ", anchor)
    print("Sentence | Similarity Score")
    for idx in ranked_indices:
        print(f"{candidates[idx]} |  {similarities[idx]:.3f} ")

In [29]:
query = "ሰማይ ምን አይነት ቀለም ነው?"
docs = [
    "ሰማይ ሰማያዊ ቀለም አለው።" ,
    "እኔ ምሳ እንጀራ በላሁ።" ,
    "ባሕር ምን አይነት ቀለም ነው?",
    "አየር በምድር ዙሪያ ያለ ነው።"
]

evaluate_model(model, "Original Model Performance: Sentence Similarity", query, docs)


============================== Original Model Performance: Sentence Similarity ==============================
Original Sentence:  ሰማይ ምን አይነት ቀለም ነው?
Sentence | Similarity Score
ባሕር ምን አይነት ቀለም ነው? |  0.764 
ሰማይ ሰማያዊ ቀለም አለው። |  0.540 
አየር በምድር ዙሪያ ያለ ነው። |  0.442 
እኔ ምሳ እንጀራ በላሁ። |  0.126 


In [40]:
loss = losses.MatryoshkaLoss(
    model,
    losses.TripletLoss(model, triplet_margin=0.5),
    matryoshka_dims=[768, 512, 256],
    matryoshka_weights=[0.5, 0.4, 0.2]
  )

evaluator = evaluation.TripletEvaluator(
    anchors=val_data["anchor"],
    positives=val_data["positive"],
    negatives=val_data["negative"],
)


In [43]:
args = SentenceTransformerTrainingArguments(
    output_dir="amharic-retrieval-ft",
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Leverages T4's 16GB VRAM
    fp16=True,
    learning_rate=1e-6,
    warmup_ratio=0.08,
    weight_decay=0.05,
    logging_dir="logs",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=300,
    save_strategy="epoch",
    report_to="tensorboard",
)

In [44]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    loss=loss,
    evaluator=evaluator, # let's keep the embedding similary evaluation away for now (atleast I get a hung of it)
)

In [45]:
trainer.train()

Step,Training Loss,Validation Loss,Cosine Accuracy
300,0.220100,0.262586,0.888095
600,0.145300,0.228679,0.884921
900,0.123100,0.213432,0.888889
1200,0.112800,0.207012,0.890476
1500,0.105300,0.202323,0.889683


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1632, training_loss=0.15497594268298617, metrics={'train_runtime': 866.6959, 'train_samples_per_second': 30.097, 'train_steps_per_second': 1.883, 'total_flos': 0.0, 'train_loss': 0.15497594268298617, 'epoch': 3.0})

In [76]:
model_4 = SentenceTransformer("/content/amharic-retrieval-ft/checkpoint-1188")

In [46]:
from sentence_transformers import evaluation

# Create a TripletEvaluator for the test set
test_evaluator = evaluation.TripletEvaluator(
    anchors=test_data["anchor"],
    positives=test_data["positive"],
    negatives=test_data["negative"],
    name="TestTripletEvaluator"
)

# Evaluate the model on the test dataset
test_score = test_evaluator(model)
print(f"Test Triplet Evaluation Score: {test_score['TestTripletEvaluator_cosine_accuracy']:.4f}")

Test Triplet Evaluation Score: 0.8750


In [78]:
# Evaluate the model on the test dataset
test_score = test_evaluator(model_4)
print(f"Test Triplet Evaluation Score: {test_score['TestTripletEvaluator_cosine_accuracy']:.4f}")

Test Triplet Evaluation Score: 0.8750


In [47]:
model_name = "xlm-r-retrieval-am-v2"
model.save("model/xlm-r-retrieval-am-v2")

In [54]:
tokenizer = model.tokenizer
tokenizer.save_pretrained("model/xlm-r-retrieval-am-v2")

('model/xlm-r-retrieval-am-v2/tokenizer_config.json',
 'model/xlm-r-retrieval-am-v2/special_tokens_map.json',
 'model/xlm-r-retrieval-am-v2/sentencepiece.bpe.model',
 'model/xlm-r-retrieval-am-v2/added_tokens.json',
 'model/xlm-r-retrieval-am-v2/tokenizer.json')

In [55]:
# prompt: copy "model/am-xlmr-ft-v-1.0" to my drive

import shutil
import os

# Assuming 'model/am-xlmr-ft-v-1.0' exists in your current Colab environment
source_dir = f"model/{model_name}"
destination_dir = f"/content/drive/MyDrive/models/{model_name}"  # Specify the destination path in your Google Drive

if os.path.exists(destination_dir):
    shutil.rmtree(destination_dir)

try:
  shutil.copytree(source_dir, destination_dir)
  print(f"Successfully copied '{source_dir}' to '{destination_dir}'")
except FileExistsError:
  print(f"Destination directory '{destination_dir}' already exists.  Skipping copy.")
except FileNotFoundError:
    print(f"Source directory '{source_dir}' not found. Please ensure the model directory exists.")
except Exception as e:
  print(f"An error occurred: {e}")

Successfully copied 'model/xlm-r-retrieval-am-v2' to '/content/drive/MyDrive/models/xlm-r-retrieval-am-v2'


In [56]:
evaluate_model(model, "Fine-Tuned Performance: Sentence Similarity", query, docs)


============================== Fine-Tuned Performance: Sentence Similarity ==============================
Original Sentence:  ሰማይ ምን አይነት ቀለም ነው?
Sentence | Similarity Score
ሰማይ ሰማያዊ ቀለም አለው። |  0.748 
ባሕር ምን አይነት ቀለም ነው? |  0.646 
አየር በምድር ዙሪያ ያለ ነው። |  0.634 
እኔ ምሳ እንጀራ በላሁ። |  0.077 


In [77]:
evaluate_model(model_4, "Fine-Tuned Performance: Sentence Similarity", query, docs)


============================== Fine-Tuned Performance: Sentence Similarity ==============================
Original Sentence:  ሰማይ ምን አይነት ቀለም ነው?
Sentence | Similarity Score
ሰማይ ሰማያዊ ቀለም አለው። |  0.871 
አየር በምድር ዙሪያ ያለ ነው። |  0.698 
ባሕር ምን አይነት ቀለም ነው? |  0.583 
እኔ ምሳ እንጀራ በላሁ። |  0.078 


In [80]:
model_4.push_to_hub("xlm-r-retrieval-am-v5")
# model.tokenizer.push_to_hub("xlm-r-retrieval-am-v4")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'https://huggingface.co/abdulmunimjemal/xlm-r-retrieval-am-v5/commit/da866785159fe55446af59c6760869751370795b'